In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 체점을 위해 임의성을 사용하는 numpy 도구들의 결과가 일정하게 나오도록 해준다
np.random.seed(5)


#나머지는 저희가 경사 하강 함수를 작성하면서 사용하는 함수들인데요. cost는 손실을 계산해 주는 함수, 
#initialize는 임의로 유저 취향과 영화 속성 행렬을 초기화해주는 함수입니다.
#predict 함수는 처음 보셨을 텐데요. 그냥 유저 취향과 영화 속성 행렬들을 받아서 곱해주는 (예측 값들을 계산해 주는) 함수입니다.


def predict(Theta, X):
    """유저 취향과 상품 속성을 곱해서 예측 값을 계산하는 함수"""
    return Theta @ X


def cost(prediction, R):
    """행렬 인수분해 알고리즘의 손실을 계산해주는 함수"""
    return np.nansum((prediction - R)**2)


def initialize(R, num_features):
    """임의로 유저 취향과 상품 속성 행렬들을 만들어주는 함수"""
    num_users, num_items = R.shape
    
    Theta = np.random.rand(num_users, num_features)
    X = np.random.rand(num_features, num_items)
    
    return Theta, X

#마지막은 저희가 구현할 gradient_descent 함수입니다. 
#파라미터로는 평점 데이터 행렬 R, 유저 취향 행렬 Theta, 영화 속성 행렬 X, 경사 하강 횟수 iteration, 학습률 alpha, 
#그리고 정규화 상수 lambda_를 받습니다. 이 파라미터들을 이용해서 유저 취향 행렬과 영화 속성 행렬을 업데이트하죠.

def gradient_descent(R, Theta, X, iteration, alpha, lambda_):
    """행렬 인수분해 경사 하강 함수"""
    #가장 먼저 유저 데이터, 영화 데이터, 속성의 개수를 파악합니다. 그리고 경사 하강을 할 때마다 손실을 계산할 건데요. 
    #이걸 저장할 파이썬 리스트를 만들어 줍니다 (costs).
    num_user, num_items = R.shape
    num_features = len(X)
    costs = []

#그다음은 경사 하강을 하고 싶은 만큼 반복문을 돕니다. 한 번 경사 하강을 할 때마다 예측 값을 계산하고 (prediction), 
#원소 별 예측 값과 실제 값의 오차를 저장하는 행렬을 계산하고 (error) 마지막으로는 손실을 costs 리스트에 추가해 줍니다.
#마지막 부분은 그냥 모든 유저, 모든 영화 데이터, 모든 속성에 대해서 다 도는 반복문인데요.
#if not np.isnan(R[i][j]): 실제 데이터 평점이 없는 값들에 대해서는 건너뛰어 주는 거 보이시죠? 

    for _ in range(iteration):
        prediction = predict(Theta, X)
        error = prediction - R
        costs.append(cost(prediction, R))

        for i in range(num_user):
            for j in range(num_items):
                if not np.isnan(R[i][j]):
                    for k in range(num_features):
                        # 아래 코드를 채워 넣으세요.
                        Theta[i][k] -= alpha * (np.nansum(error[i, :]*X[k, :]) + lambda_*Theta[i][k]) 
                        X[k][j] -= alpha * (np.nansum(error[:, j]*Theta[:, k]) + lambda_*X[k][j])
                        
    return Theta, X, costs


#----------------------실행(채점) 코드----------------------
# 평점 데이터를 가지고 온다
RATING_DATA_PATH = 'data/ratings.csv'
ratings_df = pd.read_csv(RATING_DATA_PATH, index_col='user_id')

# 평점 데이터에 mean normalization을 적용한다
for row in ratings_df.values:
    row -= np.nanmean(row)
       
R = ratings_df.values
        
Theta, X = initialize(rating, 5)  # 행렬들 초기화
Theta, X, costs = gradient_descent(R, Theta, X, 200, 0.001, 0.01)  # 경사 하강
    
# 손실이 줄어드는 걸 시각화 하는 코드 (디버깅에 도움이 됨)
# plt.plot(costs)

Theta, X